### Brandon Thimmesch, IMSE 560 TA


# Wyndor Glass Problem

In [2]:
# checking installation status of CPLEX and DOCPLEX packages
!pip install cplex --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host=files.pythonhosted.org

!pip install docplex --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host=files.pythonhosted.org

     ---------------------------------------- 0.0/26.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/26.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/26.4 MB ? eta -:--:--
     --------------------------------------- 0.0/26.4 MB 330.3 kB/s eta 0:01:20
     --------------------------------------- 0.1/26.4 MB 365.7 kB/s eta 0:01:13
     --------------------------------------- 0.1/26.4 MB 602.4 kB/s eta 0:00:44
      --------------------------------------- 0.3/26.4 MB 1.6 MB/s eta 0:00:17
      --------------------------------------- 0.5/26.4 MB 2.0 MB/s eta 0:00:14
     - -------------------------------------- 0.7/26.4 MB 2.0 MB/s eta 0:00:13
     - -------------------------------------- 0.8/26.4 MB 2.2 MB/s eta 0:00:12
     - -------------------------------------- 1.0/26.4 MB 2.4 MB/s eta 0:00:11
     - -------------------------------------- 1.1/26.4 MB 2.4 MB/s eta 0:00:11
     -- ------------------------------------- 1.3/26.4 MB 2.6 MB/s eta

## Importing Libraries

In [4]:
import sys
import docplex.mp as mp
from docplex.mp.model import Model

## Basic Model

In [18]:
# this model is a recreation of the Wyndor Glass excel model that can be found on Canvas
class LP_problem_basic: # defining LP model formulation as a function
    def __init__(self): # initializing the LP model
        
        self.model = Model(name = 'Wyndor Glass co. Product-Mix Problem -- BASIC')
    
    # defining decision variables
    def create_variables(self):
        
        # decision variable that dictates the quantity of door batches to produce
        self.Doors = self.model.continuous_var(name = 'Batches of Door', lb = 0) # lower bound = 0 (built in non-negativity constraint)
        
        # decision variable that dictates the quantity of window batches to produce
        self.Windows = self.model.continuous_var(name = 'Batches of Windows', lb = 0) # lower bound = 0 (built in non-negativity constraint))

    # defining objective function
    def create_objective(self):
        
        # objective function that maximizes total profits
        self.model.maximize(3000 * self.Doors + 5000 * self.Windows)
        # objective function sums the product of the quantities of door and window batches produced and their respective profit per batch

    # defining constraints
    def create_constraints(self):
        
        # plant 1 requires 1 hour/batch of doors produced and has 4 hours of production time available
        self.model.add_constraint(self.Doors <= 4) 
        
        # plant 2 requires 2 hours/batch of windows produced and has 12 hours of production time available
        self.model.add_constraint(2 * self.Windows <= 12) 
        
        # plant 3 requires 3 hours/batch of doors produced, 2 hours/batch of windows produced, and has 18 hours of production time available
        self.model.add_constraint(3 * self.Doors + 2 * self.Windows <= 18) 

    # defining function that prints shadow prices
    def print_shadow_prices(self):
        
        # finding dual values (shadow prices) for the constraints
        shadow_prices = self.model.dual_values(self.model.iter_constraints())
        
        # print the shadow prices for each constraint
        for y, constraint in enumerate(self.model.iter_constraints()):
            print(f"Shadow Price for Production Capacity Constraint at Plant {y + 1}: {shadow_prices[y]}")    
        
    # defining function that solves our model when called
    def solve_model(self):
        
        # solve the optimization problem
        solution = self.model.solve()
        # no need to specify a solver as DOCPLEX will determine which solver is most appropriate for the application
           
        if solution:
            print(self.model.solution)

        else:
            print("No solution found.")
            return None

In [19]:
# calling solver on our model to output a solution
solver = LP_problem_basic()
solver.create_variables()
solver.create_objective()
solver.create_constraints()
solver.solve_model()

# call function to print shadow prices
solver.print_shadow_prices()

solution for: Wyndor Glass co. Product-Mix Problem -- BASIC
objective: 36000
status: OPTIMAL_SOLUTION(2)
Batches of Door=2.000
Batches of Windows=6.000

Shadow Price for Production Capacity Constraint at Plant 1: 0
Shadow Price for Production Capacity Constraint at Plant 2: 1500.0
Shadow Price for Production Capacity Constraint at Plant 3: 1000.0


## Advanced Model
#### Makes use of variables, keys, for loops, and index references for robustness

In [20]:
# defining given constants as variables
door_profit = 3000 # $3,000 profit per batch of doors
window_profit = 5000 # $5,000 profit per batch of windows

# storing plants available
plants = ('P1','P2','P3')

# storing products available
products = ('Door','Window')

# creating dictionary array to hold production requirements (in hours) for batches of doors and windows at plants 1, 2, and 3
prod_req = {'P1':{'Door':1,'Window':0}, # requires 1 hour/batch of doors at plant 1, no window production capacity at plant 1
            'P2':{'Door':0,'Window':2}, # requires 2 hours/batch of windows at plant 2, no door production capacity at plant 2
            'P3':{'Door':3,'Window':2}} # requires 3 hours/batch of doors at plant 3, requires 2 hours/batch of doors at plant 3

# creating dictionary to hold production capacity (in hours) at plants 1, 2, and 3
prod_cap = {'P1':4,'P2':12,'P3':18} # plants 1, 2, and 3 have 4, 12, and 18 production hours available respectively

# this model is a recreation of the Wyndor Glass excel model that can be found on Canvas
class LP_problem_adv: # defining LP model formulation as a function
    
    def __init__(self): # initializing the LP model
        
        self.model = Model(name = 'Wyndor Glass co. Product-Mix Problem -- ADVANCED')

    # defining decision variables
    def create_variables(self):

        # decision variable that dictates the quantity of product batches to produce
        self.Batches = self.model.continuous_var_dict(keys = products, name = 'Batches', lb = 0) 
        # uses products (tuple) as keys, lower bound = 0 (built in non-negativity constraint)

    # defining objective function
    def create_objective(self):
        
        for i in plants:
            
            # objective function that maximizes total profits
            self.model.maximize(self.model.sum(door_profit * self.Batches['Door'])
                                + self.model.sum(window_profit * self.Batches['Window']))

    # defining constraints
    def create_constraints(self):
        
        for i in plants:
            
            # defining total production time as the production time requirement (at each plant) times the number of products we produce
            total_prod_time = self.model.sum(prod_req[i][j] * self.Batches[j] for j in products)
            
            # constraining total production time at each plant to be less than the total number of production hours available at that plant
            self.model.add_constraint(total_prod_time <= prod_cap[i])
            
    # defining function that prints shadow prices
    def print_shadow_prices(self):
        
        # finding dual values (shadow prices) for the constraints
        shadow_prices = self.model.dual_values(self.model.iter_constraints())
        
        # print the shadow prices for each constraint
        for y, plant in enumerate(plants):
            print(f"Shadow Price for Production Capacity Constraint at {plant}: {shadow_prices[y]}")
                
    # defining function that solves our model when called
    def solve_model(self):
        
        # solve the optimization problem
        solution = self.model.solve() 
        # no need to specify a solver as DOCPLEX solver will determine solving methods that are most appropriate for the application
           
        if solution:
            print(self.model.solution)

        else:
            print("No solution found.")
            return None

In [21]:
# calling solver on our model to output a solution
solver = LP_problem_adv()
solver.create_variables()
solver.create_objective()
solver.create_constraints()
solver.solve_model()

# call function to print shadow prices
solver.print_shadow_prices()

solution for: Wyndor Glass co. Product-Mix Problem -- ADVANCED
objective: 36000
status: OPTIMAL_SOLUTION(2)
Batches_Door=2.000
Batches_Window=6.000

Shadow Price for Production Capacity Constraint at P1: 0
Shadow Price for Production Capacity Constraint at P2: 1500.0
Shadow Price for Production Capacity Constraint at P3: 1000.0


We can see that both methods provide results that match the solution found using solver in Excel. You may think the advanced model is overly complicated, but storing your constraint information in a data structure (such as a list, array, or pandas dataframe) will help you create more robust, future-proof models that will be easier to modify in case any provided information changes. Additionally, referencing these arrays by their indices will help you create simplified model formulations that makes use of combinatorial notation such as X_ij (for i belonging to set I, for j belonging to set J).

Please feel free to contact me by email (brandonthimmesch@ksu.edu) if you have any questions.